<a href="https://colab.research.google.com/github/ekrombouts/gcai_zuster_fietje/blob/main/notebooks/322_fietje-2-instruct_full_ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Fietje Instruct

**Author:** Eva Rombouts  
**Date:** 2024-10-14  

### Description


In [ ]:
# Cell 1: Install necessary libraries
!pip install -q transformers datasets

In [ ]:
# Cell 2: Import required libraries and mount Google Drive
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, Dataset, DatasetDict
import time
from google.colab import runtime, drive, userdata

# Mount Google Drive
drive.mount('/content/drive')
HF_TOKEN = userdata.get('HF_TOKEN')


In [ ]:
print(setup.hf_token)

In [ ]:
# Cell 3: Load pre-trained model and tokenizer
path_hf_sampc = "ekrombouts/Gardenia_instruct_dataset"
# model_name = "BramVanroy/fietje-2-instruct"
model_name = "ekrombouts/zuster_fietje"
model_finetuned = "zuster_fietje"
commit_message = "Epoch 2"

# Load the model. Using bfloat16 gives lower precision but saves memory
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    token=HF_TOKEN
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token = HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Print memory footprint of the model
print(f"Memory footprint: {model.get_memory_footprint() / 1e9} GB")


In [ ]:
# Cell 4: Load dataset
dataset = load_dataset(path_hf_sampc, token=HF_TOKEN)
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [ ]:
train_dataset

In [ ]:
def create_prompt(row: dict, add_response: bool = True) -> str:
    """
    Generates a prompt based on the input data in 'row'.

    Args:
        row (dict): A dictionary containing 'context', 'instruction', and optionally 'response'.
        full (bool): If True, the prompt will include the 'response'.
                     If False, only 'context' and 'instruction' will be included.

    Returns:
        str: The generated prompt in text format.
    """
    # Base prompt (without response)
    prompt = f"""Context:
{row['context']}

Instructie:
{row['instruction']}

Antwoord:
"""

    # Append response if 'add_response' is True
    if add_response:
        prompt += f"{row['response']}\n"

    return prompt

In [ ]:
import random
row = random.choice(train_dataset)
prompt = create_prompt(row, True)
print(prompt)


In [ ]:
# Cell 6: Show the results of the untrained model

import random
row = random.choice(train_dataset)
prompt = create_prompt(row, add_response=False)
print(prompt)


# Display generated response and actual response
print("\nREFERENCE RESPONSE:")
print(row['response'])
print(f"{100*'-'}\n")

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
attention_mask = tokenizer(prompt, return_tensors="pt", padding=True).attention_mask.to(model.device)

# Enable cache and set model to evaluation mode
model.config.use_cache = True
model.eval()

# Generate output
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=150,
    do_sample=True,
    top_p=0.95,
    top_k=50,
    temperature=0.7,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

# Decode generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_response = generated_text[len(prompt):].strip()

print("GENERATED RESPONSE:")
print(generated_response)


In [ ]:
# Cell 7: Function to tokenize dataset samples
def collate_and_tokenize(row):
    prompt = create_prompt(row)

    # Tokenize and create labels
    encoded = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=2048,
    )
    encoded["labels"] = encoded["input_ids"].clone()
    return encoded


In [ ]:
# Cell 8: Tokenize the dataset and remove unnecessary columns
columns_to_remove = ['context', 'instruction', 'response']
# Apply tokenization
tokenized_dataset_train = train_dataset.map(collate_and_tokenize, batched=True, batch_size=1, remove_columns=columns_to_remove)
tokenized_dataset_val = val_dataset.map(collate_and_tokenize, batched=True, batch_size=1, remove_columns=columns_to_remove)


In [ ]:
# Cell 9: Print trainable parameters in the model
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(model)


In [ ]:
# Cell 10: Enable gradient checkpointing and set model to training mode
model.gradient_checkpointing_enable()
model.train()

In [ ]:
# Cell 11: Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results_full',
    report_to='none',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    warmup_steps=50,
    logging_dir='/content/drive/MyDrive/logs',
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    bf16=True,
    learning_rate=5e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True
)

In [ ]:
# Cell 12: Initialize Trainer with EarlyStoppingCallback and disable cache for training

# Disable cache for training
model.config.use_cache = False

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Add early stopping callback
)

In [ ]:
# Cell 13: Train the model and measure training time
start_time = time.time()  # Start time
trainer.train()  # Start training
end_time = time.time()  # End time

training_time = end_time - start_time  # Total training time
print(f"Training completed in {training_time} seconds.")


In [ ]:
# Cell 14: Push trained model and tokenizer to Hugging Face Hub
model.push_to_hub(
    model_finetuned,
    commit_message=commit_message,
    private=True,
    token = HF_TOKEN)

tokenizer.push_to_hub(
    model_finetuned,
    commit_message=commit_message,
    token=HF_TOKEN)


In [ ]:
# Cell 15: Stop Colab runtime (if applicable)
runtime.unassign()